In [2]:
!pip install scikit-image
!pip install Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 62.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.4/226.4 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 21.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from skimage import feature, measure, draw
import os
import logging
import gc
from tqdm import tqdm

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

Image.MAX_IMAGE_PIXELS = None

class ThemisAnalyzer:
    def __init__(self, local_dir='/kaggle/input/mars-crater-pgm'):
        self.resolution_km = 0.1  # ~100 m/pixel, in km
        self.elevation_scale = 7 / 1020  # Max 7 km depth over 0-255 range
        self.local_dir = local_dir
        
    def get_tile_path(self, lat, lon):
        if lat < 0:
            filename = f"lat{lat}_lon{lon:03d}.pgm"
        else:
            filename = f"lat{lat}_lon{lon:03d}.pgm"
        local_path = os.path.join(self.local_dir, filename)
        if os.path.exists(local_path):
            logger.info(f"Found file: {local_path}")
            return local_path
        else:
            logger.error(f"File not found: {local_path}")
            return None

    def process_tile_section(self, image_array, start_row=0, rows=1000):
        try:
            section = image_array[start_row:start_row + rows, :]
            section_resized = section[::2, ::2]  # 50% resolution for larger craters
            image_normalized = (section_resized - np.min(section_resized)) / (np.max(section_resized) - np.min(section_resized))
            
            edges = feature.canny(image_normalized, sigma=5)  # Larger sigma for bigger features
            contours = measure.find_contours(edges, 0.5)
            
            craters = []
            for contour in contours:
                contour[:, 0] = contour[:, 0] * 2 + start_row  # Scale back Y
                contour[:, 1] *= 2  # Scale back X
                mask = np.zeros_like(edges, dtype=bool)
                rr, cc = draw.polygon(contour[:, 0], contour[:, 1], mask.shape)
                mask[rr, cc] = True
                
                props = measure.regionprops(mask.astype(int))
                if props:
                    area = props[0].area
                    perimeter = props[0].perimeter
                    diameter_km = np.sqrt(4 * area / np.pi) * self.resolution_km * 2  # Adjusted for 50% downsampling
                    
                    if diameter_km >= 1.0:
                        rim_coords = list(zip(contour[:, 0].astype(int), contour[:, 1].astype(int)))
                        inside_coords = list(zip(rr, cc))
                        rim_elevations = [section_resized[y, x] for y, x in rim_coords if 0 <= y < section_resized.shape[0] and 0 <= x < section_resized.shape[1]]
                        inside_elevations = [section_resized[y, x] for y, x in inside_coords if 0 <= y < section_resized.shape[0] and 0 <= x < section_resized.shape[1]]
                        
                        if rim_elevations and inside_elevations:
                            max_rim = np.max(rim_elevations)
                            min_inside = np.min(inside_elevations)
                            depth_km = (max_rim - min_inside) * self.elevation_scale * 2  # Depth in km
                        else:
                            depth_km = 0.0

                        craters.append({
                            'diameter_km': diameter_km,
                            'depth_km': depth_km,
                            'circularity': 4 * np.pi * area / (perimeter ** 2),
                            'center_x': int(np.mean(contour[:, 1])),
                            'center_y': int(np.mean(contour[:, 0])),
                            'confidence': min(1.0, 4 * np.pi * area / (perimeter ** 2))
                        })
            return craters
        except Exception as e:
            logger.error(f"Error processing section at row {start_row}: {e}")
            return []

    def analyze_tile(self, lat, lon):
        image_path = self.get_tile_path(lat, lon)
        if not image_path:
            logger.error(f"Skipping lat={lat}, lon={lon}")
            return None, None, None
            
        logger.info(f"Loading: {os.path.basename(image_path)}")
        with Image.open(image_path) as img:
            width, height = img.size
            img_resized = img.resize((width // 2, height // 2), Image.Resampling.LANCZOS)  # 50% resolution
            image_array = np.array(img_resized, dtype=np.float32)
            total_rows = image_array.shape[0]
        
        section_size = 1000
        all_craters = []
        
        logger.info("Detecting craters...")
        for start_row in tqdm(range(0, total_rows, section_size), desc="Processing sections"):
            section_craters = self.process_tile_section(image_array, start_row, min(section_size, total_rows - start_row))
            all_craters.extend(section_craters)
            gc.collect()
        
        crater_df = pd.DataFrame(all_craters)
        stats = {
            'latitude': lat,
            'longitude': lon,
            'total_craters': len(crater_df),
            'mean_diameter': crater_df['diameter_km'].mean() if not crater_df.empty else 0,
            'median_diameter': crater_df['diameter_km'].median() if not crater_df.empty else 0,
            'min_diameter': crater_df['diameter_km'].min() if not crater_df.empty else 0,
            'max_diameter': crater_df['diameter_km'].max() if not crater_df.empty else 0,
            'mean_depth': crater_df['depth_km'].mean() if not crater_df.empty else 0,
            'median_depth': crater_df['depth_km'].median() if not crater_df.empty else 0,
            'min_depth': crater_df['depth_km'].min() if not crater_df.empty else 0,
            'max_depth': crater_df['depth_km'].max() if not crater_df.empty else 0
        }
        
        logger.info("Creating visualization...")
        fig, ax = plt.subplots(figsize=(6, 6))
        image_small = image_array[::2, ::2]
        image_normalized = (image_small - np.min(image_small)) / (np.max(image_small) - np.min(image_small))
        ax.imshow(image_normalized, cmap='gray')
        
        scale_factor = image_small.shape[0] / total_rows
        for _, crater in crater_df.iterrows():
            circle = plt.Circle(
                (crater['center_x'] * scale_factor, crater['center_y'] * scale_factor),
                crater['diameter_km'] / (2 * self.resolution_km) * scale_factor,
                fill=False,
                color='red',
                alpha=crater['confidence']
            )
            ax.add_patch(circle)
            
        ax.set_title(f'Craters (Lat {lat}, Lon {lon})')
        return crater_df, stats, fig

if __name__ == "__main__":
    logger.info("Checking disk space...")
    os.system("df -h /kaggle/working")
    
    analyzer = ThemisAnalyzer(local_dir='/kaggle/input/mars-crater-pgm')
    
    tiles_to_process = [
        {'lat': -30, 'lon': 60},
        {'lat': 30, 'lon': 0},
        {'lat': 30, 'lon': 300}
    ]
    
    logger.info("Listing files in input directory...")
    os.system("ls /kaggle/input/mars-crater-pgm")
    
    # Process tiles and collect stats
    all_stats = []
    for tile in tiles_to_process:
        lat = tile['lat']
        lon = tile['lon']
        try:
            logger.info(f"Analyzing lat={lat}, lon={lon}")
            craters, stats, figure = analyzer.analyze_tile(lat, lon)
            
            if craters is not None:
                logger.info("\nCrater Statistics:")
                for key, value in stats.items():
                    if isinstance(value, float):
                        logger.info(f"{key}: {value:.2f} km")
                    else:
                        logger.info(f"{key}: {value}")
                
                output_base = f"/kaggle/working/themis_lat{lat}_lon{lon:03d}"
                craters.to_csv(f'{output_base}_craters.csv', index=False)
                figure.savefig(f'{output_base}_detection.png', dpi=100)
                plt.close(figure)
                logger.info(f"Saved {output_base}_craters.csv and {output_base}_detection.png")
                
                all_stats.append(stats)
            
            del craters, stats, figure
            gc.collect()
            logger.info("Memory cleared")
            
        except Exception as e:
            logger.error(f"Error processing tile lat={lat}, lon={lon}: {e}")
            continue

    # Aggregate mean stats with depth
    if all_stats:
        stats_df = pd.DataFrame(all_stats)
        stats_df.to_csv('/kaggle/working/themis_stats_summary.csv', index=False)
        logger.info("Saved summary stats to /kaggle/working/themis_stats_summary.csv")

        combined_craters = pd.concat([pd.read_csv(f'/kaggle/working/themis_lat{tile["lat"]}_lon{tile["lon"]:03d}_craters.csv') for tile in tiles_to_process], ignore_index=True)
        mean_stats = {
            'total_craters': len(combined_craters),
            'mean_diameter_km': combined_craters['diameter_km'].mean(),
            'median_diameter_km': combined_craters['diameter_km'].median(),
            'min_diameter_km': combined_craters['diameter_km'].min(),
            'max_diameter_km': combined_craters['diameter_km'].max(),
            'mean_depth_km': combined_craters['depth_km'].mean(),
            'median_depth_km': combined_craters['depth_km'].median(),
            'min_depth_km': combined_craters['depth_km'].min(),
            'max_depth_km': combined_craters['depth_km'].max()
        }
        
        logger.info("\nAggregated Mean Statistics Across All Tiles (kilometers):")
        for key, value in mean_stats.items():
            if isinstance(value, float):
                logger.info(f"{key}: {value:.2f} km")
            else:
                logger.info(f"{key}: {value}")
        
        mean_stats_df = pd.DataFrame([mean_stats])
        mean_stats_df.to_csv('/kaggle/working/mean_stats_summary.csv', index=False)
        logger.info("Saved aggregated mean stats to /kaggle/working/mean_stats_summary.csv")
    else:
        logger.error("No tiles processed successfully")

2025-02-21 08:56:05,195 - INFO - Checking disk space...


Filesystem      Size  Used Avail Use% Mounted on
/dev/loop1       20G   76K   20G   1% /kaggle/working


2025-02-21 08:56:05,206 - INFO - Listing files in input directory...


lat-30_lon060.pgm
lat30_lon000.pgm
lat30_lon300.pgm


2025-02-21 08:56:05,233 - INFO - Analyzing lat=-30, lon=60
2025-02-21 08:56:05,234 - INFO - Found file: /kaggle/input/mars-crater-pgm/lat-30_lon060.pgm
2025-02-21 08:56:05,234 - INFO - Loading: lat-30_lon060.pgm
2025-02-21 08:56:15,878 - INFO - Detecting craters...
Processing sections: 100%|██████████| 9/9 [08:50<00:00, 58.96s/it]
2025-02-21 09:05:06,525 - INFO - Creating visualization...
2025-02-21 09:05:08,375 - INFO - 
Crater Statistics:
2025-02-21 09:05:08,376 - INFO - latitude: -30
2025-02-21 09:05:08,376 - INFO - longitude: 60
2025-02-21 09:05:08,376 - INFO - total_craters: 453
2025-02-21 09:05:08,377 - INFO - mean_diameter: 1.84 km
2025-02-21 09:05:08,377 - INFO - median_diameter: 1.73 km
2025-02-21 09:05:08,378 - INFO - min_diameter: 1.03 km
2025-02-21 09:05:08,378 - INFO - max_diameter: 4.14 km
2025-02-21 09:05:08,378 - INFO - mean_depth: 1.4935635328292847
2025-02-21 09:05:08,379 - INFO - median_depth: 1.358823537826538
2025-02-21 09:05:08,380 - INFO - min_depth: 0.1921568661

2025-02-21 03:24:49,689 - INFO - Loaded /kaggle/working/themis_lat-30_lon060_craters.csv with 1856 craters
2025-02-21 03:24:49,694 - INFO - Loaded /kaggle/working/themis_lat30_lon000_craters.csv with 1651 craters
2025-02-21 03:24:49,698 - INFO - Loaded /kaggle/working/themis_lat30_lon300_craters.csv with 1696 craters
2025-02-21 03:24:49,701 - INFO - 
Aggregated Mean Statistics Across All Tiles (kilometers):
2025-02-21 03:24:49,702 - INFO - total_craters: 5203
2025-02-21 03:24:49,702 - INFO - mean_diameter_km: 3.98 km
2025-02-21 03:24:49,703 - INFO - median_diameter_km: 3.29 km
2025-02-21 03:24:49,703 - INFO - min_diameter_km: 1.01 km
2025-02-21 03:24:49,704 - INFO - max_diameter_km: 18.13 km
2025-02-21 03:24:49,704 - INFO - mean_depth_km: 44.59 km
2025-02-21 03:24:49,705 - INFO - median_depth_km: 42.40 km
2025-02-21 03:24:49,705 - INFO - min_depth_km: 0.00 km
2025-02-21 03:24:49,706 - INFO - max_depth_km: 102.00 km
2025-02-21 03:24:49,708 - INFO - Saved aggregated stats to /kaggle/work